In [27]:
import os
import pandas as pd
from newsapi import NewsApiClient
from dotenv import load_dotenv
import requests

load_dotenv()

nd_api_key = os.getenv("NEWSDATA_API_KEY")
na_api_key = os.getenv("NEWSAPI_API_KEY")

def get_data_from_newsdata_api():
 params = {
     "apiKey" : nd_api_key,
     "q": "pegasus",
     "language": "en"
 }

 response = requests.get("https://newsdata.io/api/1/news", params=params)
 if response.status_code == 200:
        data = response.json()
        return data
 else:
        print("Error fetching data. Status code:", response.status_code)
        return None

def get_data_from_newsapi_api():
    newsapi = NewsApiClient(api_key=na_api_key)

    newsapi_articles = newsapi.get_everything(q='bitcoin',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2023-07-13',
                                      language='en',
                                      sort_by='relevancy')

    return newsapi_articles

na_df = pd.DataFrame(get_data_from_newsapi_api()["articles"])
nd_df = pd.DataFrame(get_data_from_newsdata_api()["results"])


